In [3]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [5]:
import pickle

# Update the file path to the correct one
file_path = '/content/drive/MyDrive/EmotionAnalysisProject-main/static/model/model.pickle'

# Load the pickled model
with open(file_path, 'rb') as f:
    model = pickle.load(f)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
# Replace this with the actual absolute path to your file
file_path = '/content/drive/MyDrive/EmotionAnalysisProject-main/static/model/corpora/stopwords/english.txt'

try:
    with open(file_path, 'r') as file:
        sw = file.read().splitlines()
    print("Stopwords loaded successfully.")
except FileNotFoundError:
    print("Error: The specified file does not exist.")


Error: The specified file does not exist.


In [11]:
file_path = '/content/drive/MyDrive/EmotionAnalysisProject-main/static/model/vocabulary.txt'

try:
    vocab = pd.read_csv(file_path, header=None)
    tokens = vocab[0].tolist()
    print("Vocabulary loaded successfully.")
except FileNotFoundError:
    print("Error: The specified file does not exist.")


Vocabulary loaded successfully.


In [12]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [34]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [33]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []

    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1

        vectorized_lst.append(sentence_lst)

    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)

    return vectorized_lst_new

In [15]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [53]:
import pickle
import pandas as pd
import numpy as np
import re
from nltk.stem import PorterStemmer

# Update the file paths to the correct ones
model_file_path = '/content/drive/MyDrive/EmotionAnalysisProject-main/static/model/model.pickle'
sw_file_path = '/content/drive/MyDrive/EmotionAnalysisProject-main/static/model/corpora/stopwords/english.txt'
vocab_file_path = '/content/drive/MyDrive/EmotionAnalysisProject-main/static/model/vocabulary.txt'

# Load the pickled model
with open(model_file_path, 'rb') as f:
    model = pickle.load(f)

# Load stopwords
try:
    with open(sw_file_path, 'r') as file:
        sw = file.read().splitlines()
    print("Stopwords loaded successfully.")
except FileNotFoundError:
    print("Error: The specified stopwords file does not exist.")

# Load vocabulary
try:
    vocab = pd.read_csv(vocab_file_path, header=None)
    tokens = vocab[0].tolist()
    print("Vocabulary loaded successfully.")
except FileNotFoundError:
    print("Error: The specified vocabulary file does not exist.")

ps = PorterStemmer()

def preprocessing(txt, stopwords):
    data = pd.DataFrame([txt], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

def vectorizer(txt, vocabulary):
    vectorized_lst = []

    sentence_lst = np.zeros(len(vocabulary))

    for i in range(len(vocabulary)):
        if vocabulary[i] in txt:
            sentence_lst[i] = 1

    vectorized_lst.append(sentence_lst)

    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)

    return vectorized_lst_new

def get_prediction(prediction_prob):
    if prediction_prob >= 0.5:
        return 'negative'
    else:
        return 'positive'

txt = "highly recommanded"
preprocessed_txt = preprocessing(txt, sw)
vectorized_txt = vectorizer(preprocessed_txt.iloc[0].split(), tokens)  # Access the text using iloc
prediction_prob = model.predict(vectorized_txt)
prediction = get_prediction(prediction_prob[0])  # Assuming prediction_prob is a single value
print(prediction)


Error: The specified stopwords file does not exist.
Vocabulary loaded successfully.
negative


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
